In [70]:
# Import Dependencies

import pandas as pd
from bs4 import BeautifulSoup as soup
from splinter import Browser

In [71]:
# Find path to our chrome browser

!which chromedriver

/usr/local/bin/chromedriver


In [72]:
# Set path to browser & start splinter

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

In [106]:
# Open webpage

url = 'http://quotes.toscrape.com/'
browser.visit(url)

In [166]:
# Create BS object

html = browser.html
soup_obj = soup(html, 'html.parser')

In [63]:
# Find first quote text

first_quote = soup_obj.find("span", class_="text").text
first_quote

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [174]:
# Find first quote tags [commented out code will return 1st tag]

first_block = soup_obj.find('div', class_="quote")
for tag in first_block:
    tags = tag.find('a', class_="tag").text
    print(tags)

# first_quote_tags = soup_obj.find("a", class_="tag").text
# first_quote_tags

AttributeError: 'NavigableString' object has no attribute 'span'

In [65]:
# Find first quote author name

first_quote_author = soup_obj.find("small", class_="author").text
first_quote_author

'Albert Einstein'

In [86]:
# Find first quote author born & description

find_author = browser.find_by_text('(about)')
find_author.click()

In [97]:
author_date = soup_obj.find('span', class_="author-born-date").text
author_location = soup_obj.find('span', class_="author-born-location").text
print(author_date)
print(author_location)

March 14, 1879
in Ulm, Germany


In [105]:
author_descrp = soup_obj.find('div', class_="author-description").text
author_descrp

'\n        In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining the photoelectric effect, the basis of electronics, earned him the Nobel Prize in 1921. His first paper on Special Relativity Theory, also published in 1905, changed the world. After the rise of the Nazi party, Einstein made Princeton his permanent home, becoming a U.S. citizen in 1940. Einstein, a pacifist during World War I, stayed a firm proponent of social justice and responsibility. He chaired the Emergency Committee of Atomic Scientists, which organized to alert the public to the dangers of atomic warfare.At a symposium, he advised: "In their struggle for the ethical good, teachers of religion must have the stature to give up the doctrine of a personal God, that is, give up that source of fear and hope which in the past placed such vast power in the hands of priests. In their labors they will have to avail themselves of those forces w